In [ ]:
import pandas as pd
import datetime
import os
import sys
import csv

In [ ]:
def data_Validation_2(df_data, data_name, date):

    if df_data is None:
        return print('\nThis is empty data\t' + data_name)

    len_df_data = len(df_data)

    min_data = df_data['UnixTime'].min()
    max_data = df_data['UnixTime'].max()

    try:
        df_data = df_data.astype('int')
        df_data = df_data.astype({'UnixTime':'int'})
        df_data = df_data.sort_values('UnixTime')
        df_data = df_data.reset_index(drop=True)
        
    except OverflowError:
        print("ERROR OverflowError, " + date + ", " + data_name)
        pass

    return (len_df_data - df_data.duplicated().sum())

In [ ]:
csv.field_size_limit(10000000)

In [ ]:
# 센서 리스트 선언
sensors = ['mAcc', 'mGyr', 'mPre', 'mLi', 'wAcc', 'wGyr', 'wPre', 'wHR']

# sid = int(input("Input SID"))
sid = ["608", "609", "610", "611", "612"]
# sid = ["608"]

for sid in sid:
    # start_Month = int(input("Input start month"))
    # start_Day = int(input("Input start day"))
    start_Month = 8
    start_Day = 5

    start_Date = datetime.datetime(2022, start_Month, start_Day)

    loop_Flag = True

    for i in range(0, 60):

        if loop_Flag == True:

            all_Instance = 0

            date_Flag = 0

            for j in range(0, 8):

                df = pd.DataFrame()

                date = start_Date.strftime("%m%d")

                csv_Folder_Root = "D:\KNIH_Val/S" + str(sid)
                val_Folder_Root = "D:\KNIH_Val/val"

                # txt_Root = "./val/S" + str(sid) + "/" + str(sid) + "_" + date + "_" + sensors[j] + ".txt"
                csv_Root = "D:\KNIH_Val/S" + str(sid) + "/" + str(sid) + "_" + date + "_" + sensors[j] + ".csv"
                val_Root = "D:\KNIH_Val/val/" + str(sid) + ".csv"

                # 디렉토리 만들기
                try:
                    if not os.path.exists(csv_Folder_Root):
                        os.makedirs(csv_Folder_Root)
                except OSError:
                    print('Error: Creating directory. ' + csv_Folder_Root)
                    
                try:
                    if not os.path.exists(val_Folder_Root):
                        os.makedirs(val_Folder_Root)
                except OSError:
                    print('Error: Creating directory. ' + val_Folder_Root)
                
                try:    

                    try:
                        if not os.path.exists(val_Root):
                            # os.makedirs(txt_Root)

                            f = open(val_Root, 'w')
                            f.write('date,mAcc,mGyr,mPre,mLi,wAcc,wGyr,wPre,wHR,All\n')
                            f.close()

                    except OSError:
                        print('Error: Creating directory. ' + val_Root)

                    f = open(val_Root, 'a')

                    if date_Flag != 1:
                        f.write(date + ',')
                        date_Flag = 1

                    try:
                        # ParserError: Too many columns specified 에러 뜨면 header=None 옵션 지우기
                        # df = pd.read_csv(csv_Root, on_bad_lines='skip', header=None)
                        # csv 읽기, on_bad_lines='skip' 해줘야 함
                        # df = pd.read_csv(csv_Root, on_bad_lines='skip')
                        df = pd.read_csv(csv_Root, on_bad_lines='skip', header=None, quoting=csv.QUOTE_MINIMAL, sep=',')
                        # df = pd.read_csv(csv_Root, error_bad_lines=False)

                        # f = open(csv_Root, 'r', encoding='utf-8')
                        # rdr = csv.reader(f)
                        # for line in rdr:
                        #     # print(line)
                        #     df.append(line)
                        # f.close()   

                    # except OverflowError:
                    #     print("ERROR OverflowError")
                    #     pass
                    #     # 10단위로 나누며 maxsize결정

                    except Exception:

                        try:
                            df = pd.read_csv(csv_Root, on_bad_lines='skip', quoting=csv.QUOTE_MINIMAL, sep=',')
                        
                        except Exception:
                            
                            # print("ERROR, ReadCSV_Error, date: " + date + ", Sen: " + sensors[j])
                            print("ERROR, DataNotFound, S" + sid  + ", Date: " + date + ", " + sensors[j])
                            f.write(str(-1) + ",")
                            continue

                        pass

                    try:
                        # UnixTime과 UnixTime_sub이 될 값만 선택하여 df를 재구성
                        df = df.iloc[:, 0:2]
                        df = df.dropna()
                        
                        df.columns = ['UnixTime', 'UnixTime_sub']
                        
                        # df 캐스팅
                        df['UnixTime'] = df['UnixTime'].astype(int)
                        df['UnixTime'] = df['UnixTime'].astype(str)
                        df['UnixTime_sub'] = df['UnixTime_sub'].astype(int)
                        df['UnixTime_sub'] = df['UnixTime_sub'].astype(str)

                        # 6자리 맞추기
                        df['UnixTime_sub'] = df['UnixTime_sub'].str.pad(width=6, side='left', fillchar='0')
                        df['UnixTime'] = df['UnixTime'] + df['UnixTime_sub']
                        df = df.drop('UnixTime_sub', axis = 1)

                        # 이상치 제거
                        index_drop = df[df['UnixTime'].str.contains('-')].index
                        df = df.drop(index_drop)

                        # 중복값 제거
                        df = df.drop_duplicates(['UnixTime'], keep = 'first', ignore_index = True)

                        val_result = data_Validation_2(df, sensors[j], date)

                        # print("[Number of instances] " + sensors[j] + "\t" +str(val_result))

                        sen_Percent = round(val_result/86400 * 100.0, 1)

                        if sen_Percent > 100:
                            sen_Percent = 100

                        f.write(str(sen_Percent) + ",")

                        all_Instance = all_Instance + int(val_result)
                    
                    except Exception as e:
                        print(e)
                        continue

                except FileNotFoundError as e:
                    print("ERROR\t003\t" + sensors[j])
                    print("\n[FileNotFoundError]\t" + csv_Root)
                    loop_Flag = False
                    pass

                rate = round(float(all_Instance) / 691200.0 * 100.0, 1)

                # df = df.reset_index()
                # print(df.index)
                del [[df]]
                df = pd.DataFrame()


                # Loop - Validation END

            print("[Average daily collection rate]\tS" + sid + "\t" + date + "\t" + str(rate) + "%")
            f.write(str(rate) + "\n")
            f.close()

            rate = 0

            start_Date = start_Date + datetime.timedelta(days=1)

    f.close()

    print("\n[DONE]")

